Wewill use 2D-convolution kernels and the OpenCV Computer Vision library to apply different blurring and sharpening techniques to an image.

##### An Introduction to Convolution Kernels in Image Processing
In image processing, a convolution kernel is a 2D matrix that is used to filter images. Also known as a convolution matrix, a convolution kernel is typically a square, MxN matrix, where both M and N are odd integers (e.g. 3×3, 5×5, 7×7 etc.). See the 3×3 example matrix given below.


A 3×3 2D convolution kernel


Such kernels can be used to perform mathematical operations on each pixel of an image to achieve a desired effect (like blurring or sharpening an image). But why would you want to blur an image? Here are two important reasons:

1. Because it reduces certain types of noise in an image. For this reason, blurring is often referred to as smoothing.
2. To remove a distracting background, you might intentionally blur portions of an image, as is done in ‘Portrait’ mode, on mobile device cameras.

Being a fundamental processing technique in Computer Vision, filtering images with kernels has many more applications.

##### How to Use Kernels to Sharpen or Blur Images
Filtering of a source image is achieved by convolving the kernel with the image. In simple terms, convolution of an image with a kernel represents a simple mathematical operation, between the kernel and its corresponding elements in the image.

Assume that the center of the kernel is positioned over a specific pixel (p), in an image.
Then multiply the value of each element in the kernel (1 in this case), with the corresponding pixel element (i.e. its pixel intensity) in the source image.

Now, sum the result of those multiplications and compute the average.
Finally, replace the value of pixel (p), with the average value you just computed.
Once you perform this operation for every pixel in the source image, using the above 3×3 kernel, the resulting filtered image will appear blurred. This is because the convolution operation with this kernel has an averaging effect, which tends to smooth or blur the image. You will soon see for yourself how the value of individual elements in a kernel dictate the nature of filtering. For example, by changing the value of the kernel elements, you can also achieve a sharpening effect. The concept is simple yet very powerful, and is therefore used in numerous image processing pipelines.


##### Applying the Identity Kernel to an Image in OpenCV
Before we describe how to implement blurring and sharpening kernels, let’s first learn about the identity kernel. The identity kernel is a square matrix, where the middle element is 1, and all other elements are zero, as shown below.

A 3×3 identity kernel

What makes an identity matrix special is that multiplying it with any other matrix will return the original matrix. Let’s now demonstrate how to use this identity kernel with OpenCV filtering functions. In this first example, we will use the above identity kernel to show that the filtering operation leaves the original image unchanged.  

Start by importing OpenCV and Numpy, as shown in the code below.

The following steps are performed in the code below:

1. Read the test image
2. Define the identity kernel, using a 3×3 NumPy array
3. Use the filter2D() function in OpenCV to perform the linear filtering operation
4. Display the original and filtered images, using imshow()
5. Save the filtered image to disk, using imwrite()

filter2D(src, ddepth, kernel)

The filter2D()function requires three input arguments:

1.The first argument is the source image
2.The second argument is ddepth, which indicates the depth of the resulting image. A value of -1 3.indicates that the final image will also have the same depth as the source image
4.The final input argument is the kernel, which we apply to the source image


##### Blurring an Image using a Custom 2D-Convolution Kernel
Next, we will demonstrate how to blur an image. Here too, we will define a custom kernel, and use the filter2D() function in OpenCV to apply the filtering operation on the source image. 

Begin by defining a 5×5 kernel, consisting of only ones. Note that we also divide the kernel by 25. Why is that? Well, before you apply any convolution to an image, using a 2D-convolution matrix, you need to ensure that all the values are normalized. This is done by dividing each element of the kernel, by the number of elements in the kernel, which in this case is 25. This ensures all values stay within the range of [0,1]. 

Now use the filter2D() function to filter the image. As you can see, filter2D() can be used to convolve an image, with any user-defined kernel.



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread('wood.jpg')

# Print error message if image is null
if image is None:
    print('Could not read image')

# Apply identity kernel
kernel1 = np.array([[0, 0, 0],
                    [0, 1, 0],
                    [0, 0, 0]])

identity = cv2.filter2D(src=image, ddepth=-1, kernel=kernel1)

# Apply blurring kernel
kernel2 = np.ones((5, 5), np.float32) / 25
blurred = cv2.filter2D(src=image, ddepth=-1, kernel=kernel2)

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Original image
axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original')

# Identity kernel result
axs[1].imshow(cv2.cvtColor(identity, cv2.COLOR_BGR2RGB))
axs[1].set_title('Identity Kernel')

# Show the plots
plt.show()



##### Blurring an Image Using OpenCV’s Built-In Function
You can also blur an image, using OpenCV’s built-in blur() function. Essentially a convenience function, use it to blur images, where you need not specifically define a kernel.  Simply specify the kernel size, using the ksize input argument, as shown in the code below. The blur function will then internally create a 5×5 blur kernel, and apply it to the source image.

The example below, which uses the blur() function will generate exactly the same output as the example above, which had used  the filter2d() function.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def add_gaussian_noise(image, mean=0, std=25):
    """Function to add Gaussian noise to the image."""
    row, col, ch = image.shape
    gaussnoise = np.random.normal(mean, std, (row, col, ch))
    scale = 1
    noisy = image + scale*gaussnoise
    noisy = np.clip(noisy, 0, 255)
    return noisy.astype(np.uint8)

def calculate_snr(original, noisy):
    """Function to calculate Signal-to-Noise Ratio (SNR)."""
    mse = np.mean((original - noisy) ** 2)
    snr = 10 * np.log10(np.mean(original ** 2) / (mse + 1e-10))
    return snr

# Read the image
image = cv2.imread('wood.jpg')

# Print error message if image is null
if image is None:
    print('Could not read image')

# Add Gaussian noise to the image
noisy_image = add_gaussian_noise(image)

# Blur the noisy image using cv2.blur
blurred_noisy_image = cv2.blur(src=noisy_image, ksize=(3, 3))

# Calculate SNR for the original, noisy input, and blurred image
snr_original = calculate_snr(image, image)
snr_noisy = calculate_snr(image, noisy_image)
snr_blurred = calculate_snr(image, blurred_noisy_image)

# Display using Matplotlib
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Original image
axs[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axs[0, 0].set_title(f'Original (SNR: {snr_original:.2f} dB)')

# Noisy image
axs[0, 1].imshow(cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB))
axs[0, 1].set_title(f'Noisy (SNR: {snr_noisy:.2f} dB)')

# Blurred image
axs[0, 2].imshow(cv2.cvtColor(blurred_noisy_image, cv2.COLOR_BGR2RGB))
axs[0, 2].set_title(f'Blurred (SNR: {snr_blurred:.2f} dB)')

# Plot SNR values
axs[1, 0].axis('off')
axs[1, 0].text(0.5, 0.5, f'SNR Original: {snr_original:.2f} dB', ha='center', va='center', fontweight='bold')
axs[1, 1].axis('off')
axs[1, 1].text(0.5, 0.5, f'SNR Noisy: {snr_noisy:.2f} dB', ha='center', va='center', fontweight='bold')
axs[1, 2].axis('off')
axs[1, 2].text(0.5, 0.5, f'SNR Blurred: {snr_blurred:.2f} dB', ha='center', va='center', fontweight='bold')

# Show the plots
plt.tight_layout()
plt.show()




##### Applying Gaussian Blurring to an Image in OpenCV
We will now apply a Gaussian blur to an image, using OpenCV. This technique uses a Gaussian filter, which performs a weighted average, as opposed to the uniform average described in the first example. In this case, the Gaussian blur weights pixel values, based on their distance from the center of the kernel. Pixels further from the center have less influence on the weighted average. The following code convolves an image, using the GaussianBlur() function in OpenCV.

GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]])

The GaussianBlur() function requires four input arguments:

1. The first argument, src, specifies the source image that you want to filter.
2. The second argument is ksize, which defines the size of the Gaussian kernel. Here, we are using a 5×5 kernel.
3. The final two arguments are sigmaX and sigmaY, which are both set to 0. These are the Gaussian kernel standard deviations, in the X (horizontal) and Y (vertical) direction. The default setting of sigmaY is zero. If you simply  set sigmaX to zero, then the standard deviations are computed from the kernel size (width and height respectively). You can also explicitly set the size of each argument to positive values greater than zero.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread('wood.jpg')

# Print error message if image is null
if image is None:
    print('Could not read image')

# Apply Gaussian blur
gaussian_blur = cv2.GaussianBlur(src=image, ksize=(5, 5), sigmaX=0, sigmaY=0)

# Display using Matplotlib
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Original image
axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original')

# Gaussian blurred image
axs[1].imshow(cv2.cvtColor(gaussian_blur, cv2.COLOR_BGR2RGB))
axs[1].set_title('Gaussian Blurred')

# Show the plots
plt.show()


##### Applying Median Blurring to an Image in OpenCV
We can also apply median blurring, using the medianBlur() function in OpenCV. In median blurring, each pixel in the source image is replaced by the median value of the image pixels in the kernel area.

medianBlur(src, ksize)

This function has just two required arguments:

The first is the source image.
The second is the kernel size, which must be an odd, positive integer.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread('wood.jpg')

# Print error message if image is null
if image is None:
    print('Could not read image')

# Apply median blur
median_blur = cv2.medianBlur(src=image, ksize=5)

# Display using Matplotlib
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Original image
axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original')

# Median-blurred image
axs[1].imshow(cv2.cvtColor(median_blur, cv2.COLOR_BGR2RGB))
axs[1].set_title('Median Blurred')

# Show the plots
plt.show()


##### Sharpening an Image Using Custom 2D-Convolution Kernels


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image = cv2.imread('wood.jpg')

# Print error message if image is null
if image is None:
    print('Could not read image')

# Define sharpening kernel
kernel3 = np.array([[0, -1, 0],
                    [-1, 5, -1],
                    [0, -1, 0]])

# Apply sharpening
sharp_img = cv2.filter2D(src=image, ddepth=-1, kernel=kernel3)

# Display using Matplotlib
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Original image
axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original')

# Sharpened image
axs[1].imshow(cv2.cvtColor(sharp_img, cv2.COLOR_BGR2RGB))
axs[1].set_title('Sharpened')

# Show the plots
plt.show()


#### Applying Bilateral Filtering to an Image in OpenCV
While blurring can be an effective way to reduce noise in an image, it is often not desirable to blur the entire image, as important details and sharp edges may be lost. In such cases, bilateral filtering can make your life easier.

This technique applies the filter selectively to blur similar intensity pixels in a neighborhood. Sharp edges are preserved, wherever possible.
It lets you control not only the spatial size of the filter, but also the degree to which the neighboring pixels are included in the filtered output. This is done, based on variation in their color intensity, and also distance from the filtered pixel.
Bilateral filtering essentially applies a 2D Gaussian (weighted) blur to the image, while also considering the variation in intensities of neighboring pixels to minimize the blurring near edges (which we wish to preserve). What this means is that the shape of the kernel actually depends on the local image content, at every pixel location.

Here’s a concrete example. Assume, you are filtering a region in an image, near an edge. A simple Gaussian blur filter would blur the edge because it lies near the filtered region (close to the center of the Gaussian filter).  But the bilateral filter can sense the edge, because it also considers differences in pixel intensities. So, it will compute a much lower weight for the pixels straddling the edge, thereby reducing their influence on the filtered region. Regions of more uniform intensity are blurred heavier, as they are not associated with strong edges.

Thankfully, OpenCV provides the bilateralFilter() function to filter images.

bilateralFilter(src, d, sigmaColor, sigmaSpace)

This function has four required arguments:

The first argument of the function is the source image.
The next argument d, defines the diameter of the pixel neighborhood used for filtering.
The next two arguments, sigmaColor and sigmaSpace define the standard deviation of the (1D) color-intensity distribution and (2D) spatial distribution respectively.
The sigmaSpace parameter defines the spatial extent of the kernel, in both the x and y directions (just like the Gaussian blur filter previously described).
The sigmaColor parameter defines the one-dimensional Gaussian distribution, which specifies the degree to which differences in pixel intensity can be tolerated.
The final (weighted) value for a pixel in the filtered image is a product of its spatial and intensity weight. Thus,

pixels that are similar and near the filtered pixel will have influence
pixels that are far away from the filtered pixel will have little influence (due to the spatial Gaussian)
pixels that have dissimilar intensities will have little influence (due to the color-intensity Gaussian), even if they are close to the center of the kernel.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Using the function bilateralFilter() where d is diameter of each...
# ...pixel neighborhood that is used during filtering.
# sigmaColor is used to filter sigma in the color space.
# sigmaSpace is used to filter sigma in the coordinate space.

image = cv2.imread('wood.jpg')

if image is None:
    print('Could not read image')

bilateral_filter = cv2.bilateralFilter(src=image, d=9, sigmaColor=75, sigmaSpace=75)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axs[0].set_title('Original')

axs[1].imshow(cv2.cvtColor(bilateral_filter, cv2.COLOR_BGR2RGB))
axs[1].set_title('Bilateral Filtering')

plt.show()


Emboss
Forming a 3D design that pops out of the surface is called Emboss. It replaces the pixel with a shadow or a highlight. 

The following kernel matrix can be used to apply to emboss filter:



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
  
# Reading the image from the disk using cv2.imread() function
# Showing the original image using matplotlib library function plt.imshow()
img = cv2.imread('wood.jpg')
plt.imshow(img)
plt.show()
  
# Apply kernel for embossing
emboss_kernel = np.array([[-1, 0, 0],
                          [0, 0, 0],
                          [0, 0, 1]])
  
# Embossed image is obtained using the variable emboss_img
# cv2.filter2D() is the function used
# src is the source of image(here, img)
# ddepth is destination depth. -1 will mean the output image will have the same depth as the input image
# kernel is used for specifying the kernel operation (here, emboss_kernel)
emboss_img = cv2.filter2D(src=img, ddepth=-1, kernel=emboss_kernel)
  
# Showing the embossed image using matplotlib library function plt.imshow()
plt.imshow(emboss_img)
plt.show()
